# CAO 
***
## Author: Fionn McCarthy

## Introduction and Background
***

Below shows the libraries imported to carry out the relevant analysis

In [1]:
# Regular expressions 
import re

# Working with HTTP 
import requests as rq

# Data frames 
import pandas as pd

# Date and time library
import datetime as dt 

# For downloading from URl
import urllib.request as urlrq

***
***
# 2021 CAO Data
***
### The link to the CAO points 2021 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2021)**.
Over the course of the assignment the data for the 2021 Leaving Certificate points changed format on the website from HTML to Excel format. For teh purpose of this analysis I think it would be good idea to show how both formats can be extracted. In this case I will first show how regular expressions was utilized to extarct the 2021 CAo data in HTML form and then show how the Excel file was converted into daat for the purpose of this analysis.

In [2]:
# Retrieve the CAO points from the URl below
CAO1 = rq.get('http://www2.cao.ie/points/l8.php')

# View the above is okay, 200 means all okay. 
CAO1 

<Response [200]>

### Get the date and time of right now, in order to use for file creation time throughout notebook.
The datetime library is usesd in order to take a back up of the data retrieved from the CAO website.The reason the datetime library is used is to back up the data each time the notebook is run as to not overwrite the data. 

In [3]:
# date and time of right now.
now = dt.datetime.now()

# Formatting now as a string.
nowstring = now.strftime('%Y%m%d_%H%M%S')

### Saving the Original Data Set 

The server advises to utilize charset 'iso-8859-1' in order to decode the but from research this charset is unable to decode \x96 as it is not defined so we use 'cp1252' instead.

In [4]:
# Setting the charset to decode with 
CAO1.encoding = 'cp1252'

In [5]:
# Setting the charset to decode with 
CAO1.encoding = 'cp1252'

# Creating a file path for the data.
path2021_html = 'data/cao2021_' + nowstring + '.html'

# Saving the original html file before going any further. 
with open(path2021_html, 'w') as f:
    f.write(CAO1.text)

### Regular expressions selects the lines in the we want to keep.

In [6]:
# The regular expression is compiled below 
re_expr = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

In [7]:
# This function will split out the points that had either an '#' or '*' associated with them.
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = '*'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    # Return an array with points, portfolio, random        
    return [points, portfolio, random]

In [8]:
# Path creation for the csv file.
path2021_csv = 'data/cao2021_csv_' + nowstring + '.csv'

Looping through lines 

In [9]:
# Keep track of how many courses we process.
no_lines = 0

# Open and write to the cvs file
with open(path2021_csv, 'w') as f:
    # Write a header row for data set
    f.write(','.join(['code', 'title', 'points_r1', 'points_r2']) + '\n')
    # Loop through lines of the response
    for line in CAO1.iter_lines():
        # Decode using cp1252
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_expr.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code, first five characters in string
            course_code = dline[:5]
            # The course title
            course_title = dline[7:57].strip()
            # Course points split on one or more spaces after character 60
            course_points = re.split('  +', dline[60:]) 
            if len(course_points) != 2:
                course_points = course_points[:2]
            # join the fields using a comma 
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')
        
print(no_lines)

949


#### The above count of 949 courses was verified by a manual count that was carried out in visual studio code on original html file.

In [10]:
# Read csv file into 2021 dataframe
df2021 = pd.read_csv(path2021_csv, encoding='cp1252')

### 2021 Data change of Format on CAO website to excel, I will show how I converted this into data to be utlized in the analysis for this assignment.

In [11]:
# Creating a file path for the excel data.
path2021_excel = 'data/cao2021_' + nowstring + '.xlsx'

In [12]:
# https://stackoverflow.com/questions/19602931/basic-http-file-downloading-and-saving-to-disk-in-python
# Save original file
urlrq.urlretrieve("http://www2.cao.ie/points/CAOPointsCharts2021.xlsx", path2021_excel) 

('data/cao2021_20211210_165508.xlsx',
 <http.client.HTTPMessage at 0x1d0442684c0>)

In [13]:
# Download and parse the excel spreadsheet
# From examining the excel file I noted that needed to skip the first 10 trows of excel file that is loaded into dataframe
df2021_2 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2021.xlsx', skiprows = 11)

In [14]:
df2021_2

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Engineering and engineering trades,Music and Instrument Technology,AL605,211,NaN,NaN,NaN,211,NaN,319,6,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Pharmacy Technician,AL630,308,NaN,NaN,NaN,308,NaN,409,6,Athlone Institute of Technology,NaN,NaN,NaN
2,Health,Dental Nursing,AL631,311,NaN,NaN,NaN,311,NaN,400,6,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Applied Science,AL632,297,NaN,NaN,NaN,297,NaN,454,6,Athlone Institute of Technology,NaN,NaN,NaN
4,Business and administration,Business,AL650,AQA,NaN,AQA,NaN,AQA,NaN,351,6,Athlone Institute of Technology,NaN,avp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,270,NaN,NaN,NaN,270,NaN,392,8,Waterford Institute of Technology,NaN,NaN,NaN
1447,Personal services,Recreation and Sport Management,WD212,262,NaN,NaN,NaN,262,NaN,304,8,Waterford Institute of Technology,NaN,NaN,NaN
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,230,NaN,230,NaN,230,NaN,361,8,Waterford Institute of Technology,NaN,avp,NaN
1449,Welfare,Early Childhood Care and Education,WD231,266,NaN,NaN,NaN,266,NaN,366,8,Waterford Institute of Technology,NaN,NaN,NaN


In [15]:
# Check of bottom value of 2021 file is correct
df2021_2.iloc[-1]

CATEGORY (ISCED Description)          Business and administration
Course Title                         Business Information Systems
Course Code                                                 WD232
R1 Points                                                     261
R1 Random                                                     NaN
R2 Points                                                     261
R2 Random                                                     NaN
EOS Points                                                    261
EOS Random                                                    NaN
EOS Midpoints                                                 329
Course Level                                                    8
HEI                             Waterford Institute of Technology
Test/Interview                                                NaN
AVP                                                           avp
v                                                             NaN
Name: 1450

In [16]:
df2021_2.dtypes

CATEGORY (ISCED Description)    object
Course Title                    object
Course Code                     object
R1 Points                       object
R1 Random                       object
R2 Points                       object
R2 Random                       object
EOS Points                      object
EOS Random                      object
EOS Midpoints                   object
Course Level                     int64
HEI                             object
Test/Interview                  object
AVP                             object
v                               object
dtype: object

In [17]:
df2021_2.columns

Index(['CATEGORY (ISCED Description)', 'Course Title', 'Course Code',
       'R1 Points', 'R1 Random', 'R2 Points ', 'R2 Random', 'EOS Points',
       'EOS Random', 'EOS Midpoints', 'Course Level', 'HEI', 'Test/Interview',
       'AVP', 'v'],
      dtype='object')

In [18]:
# https://stackoverflow.com/questions/44117326/how-can-i-remove-all-non-numeric-characters-from-all-the-values-in-a-particular
# Remove characters in columns that are non-numeric. 
# Note there is a space at end of Round 2 points col name
df2021_2['EOS Points'] = df2021_2['EOS Points'].str.extract('(\d+)', expand=False)
df2021_2['R1 Points'] = df2021_2['R1 Points'].str.extract('(\d+)', expand=False)
df2021_2['R2 Points '] = df2021_2['R2 Points '].str.extract('(\d+)', expand=False)
df2021_2['EOS Midpoints'] = df2021_2['EOS Midpoints'].str.extract('(\d+)', expand=False)

# Conver columns to floats
df2021_2['EOS Points'] = df2021_2['EOS Points'].astype(float)
df2021_2['R1 Points'] = df2021_2['R1 Points'].astype(float)
df2021_2['R2 Points '] = df2021_2['R2 Points '].astype(float)
df2021_2['EOS Midpoints'] = df2021_2['EOS Midpoints'].astype(float)

In [19]:
df2021_2.dtypes

CATEGORY (ISCED Description)     object
Course Title                     object
Course Code                      object
R1 Points                       float64
R1 Random                        object
R2 Points                       float64
R2 Random                        object
EOS Points                      float64
EOS Random                       object
EOS Midpoints                   float64
Course Level                      int64
HEI                              object
Test/Interview                   object
AVP                              object
v                                object
dtype: object

In [20]:
df2021_2

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Engineering and engineering trades,Music and Instrument Technology,AL605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Pharmacy Technician,AL630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN
2,Health,Dental Nursing,AL631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Applied Science,AL632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN
4,Business and administration,Business,AL650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,avp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,NaN,NaN
1447,Personal services,Recreation and Sport Management,WD212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,NaN,NaN
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,avp,NaN
1449,Welfare,Early Childhood Care and Education,WD231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,NaN,NaN


In [21]:
# taking first 2 characters in Course Code column and outputting to new column hei_code column of df
df2021_2['hei_code'] = df2021_2['Course Code'].str[:2]
df2021_2

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v,hei_code
0,Engineering and engineering trades,Music and Instrument Technology,AL605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
1,Health,Pharmacy Technician,AL630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
2,Health,Dental Nursing,AL631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
3,Biological and related sciences,Applied Science,AL632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
4,Business and administration,Business,AL650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Athlone Institute of Technology,NaN,avp,NaN,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,NaN,NaN,WD
1447,Personal services,Recreation and Sport Management,WD212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,NaN,NaN,WD
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,avp,NaN,WD
1449,Welfare,Early Childhood Care and Education,WD231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Waterford Institute of Technology,NaN,NaN,NaN,WD


Validated, above is correct when compared to file.

#### Save Pandas Dataframe for 2021 Excel data  

In [22]:
# Creating a file path for the pandas data.
path2021_2_csv = 'data/cao2021_2_' + nowstring + '.csv'

In [23]:
# Save pandas dataframe to folder
df2021_2.to_csv(path2021_2_csv)

***
***
# 2020 CAO Data
***
### The link to the CAO points 2020 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2020)**.

#### Save Original 2020 Data

In [24]:
# Creating a file path for the excel data.
path2020_excel = 'data/cao2020_' + nowstring + '.xlsx'

In [25]:
# https://stackoverflow.com/questions/19602931/basic-http-file-downloading-and-saving-to-disk-in-python
# Save original file
urlrq.urlretrieve("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", path2020_excel) 

('data/cao2020_20211210_165508.xlsx',
 <http.client.HTTPMessage at 0x1d044cb64f0>)

#### Load Spreadhseet using Pandas

In [26]:
# Download and parse the excel spreadsheet
# From examining the excel file I noted that needed to skip the first 10 trows of excel file that is loaded into dataframe
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows = 10)

In [27]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [29]:
df2020.dtypes

CATEGORY (i.e.ISCED description)     object
COURSE TITLE                         object
COURSE CODE2                         object
R1 POINTS                            object
R1 Random *                          object
R2 POINTS                            object
R2 Random*                           object
EOS                                  object
EOS Random *                         object
EOS Mid-point                        object
LEVEL                                 int64
HEI                                  object
Test/Interview #                     object
avp                                  object
v                                    object
Column1                             float64
Column2                             float64
Column3                             float64
Column4                             float64
Column5                             float64
Column6                             float64
Column7                             float64
Column8                         

In [30]:
# https://stackoverflow.com/questions/44117326/how-can-i-remove-all-non-numeric-characters-from-all-the-values-in-a-particular
# Remove characters in EOS column that are non-numeric. 
df2020['EOS'] = df2020['EOS'].str.extract('(\d+)', expand=False)
df2020['R1 POINTS'] = df2020['R1 POINTS'].str.extract('(\d+)', expand=False)
df2020['R2 POINTS'] = df2020['R2 POINTS'].str.extract('(\d+)', expand=False)
df2020['EOS Mid-point'] = df2020['EOS Mid-point'].str.extract('(\d+)', expand=False)

# Conver EOS column to float 
df2020['EOS'] = df2020['EOS'].astype(float)
df2020['R1 POINTS'] = df2020['R1 POINTS'].astype(float)
df2020['R2 POINTS'] = df2020['R2 POINTS'].astype(float)
df2020['EOS Mid-point'] = df2020['EOS Mid-point'].astype(float)

In [31]:
df2020.dtypes

CATEGORY (i.e.ISCED description)     object
COURSE TITLE                         object
COURSE CODE2                         object
R1 POINTS                           float64
R1 Random *                          object
R2 POINTS                           float64
R2 Random*                           object
EOS                                 float64
EOS Random *                         object
EOS Mid-point                       float64
LEVEL                                 int64
HEI                                  object
Test/Interview #                     object
avp                                  object
v                                    object
Column1                             float64
Column2                             float64
Column3                             float64
Column4                             float64
Column5                             float64
Column6                             float64
Column7                             float64
Column8                         

In [32]:
# taking first 2 characters in Course Code column and outputting to new column hei_code column of df
df2020['hei_code'] = df2020['COURSE CODE2'].str[:2]
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,hei_code
0,Business and administration,International Business,AC120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AC
1,Humanities (except languages),Liberal Arts,AC137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AC
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AD
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AD
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD
1462,Personal services,Recreation and Sport Management,WD212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD


#### Save Pandas Dataframe

In [33]:
# Creating a file path for the pandas data.
path2020_csv = 'data/cao2020_' + nowstring + '.csv'

In [34]:
# Save pandas datafram to folder
df2020.to_csv(path2020_csv)

***
***
# 2019 CAO Data
***
### The link to the CAO points 2019 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2019)**.
The level 8 data and the level 6 and 7 data are held on two different files so I will first manipulate the level 8 data and then the level 6 and 7 data. 
### Level 8 data


#### In order to reproduce the data these are the steps I followed 
1. Download the original level 8 PDF file from the CAO website (link above). 
2. Open the original file using Microsoft Word. 
3. When open in Word, save the document as a word document. 
4. Re-save the Word document for editing.
5. Delete headers and page numbers in footer.
6. Select all in document and copy. 
7. Paste into Notepad++.
8. Remove institution name lines and blank lines.
9. In column headings, replace 'COURSE AND INSTITUTION' with 'Course', save file into folder as a csv file.
10. Change backticks to apostrophes. 
11. Delete tabs at end of lines: 61, 64, 169, 171, 172, 200, 335, 352, 425, 433, 700, 701, 752, 793, 830, 837, 869 and 894. 
12. Remove tab after 'Mid' first line.
13. Remove double tab on line 37.
14. Delete tabs at end of lines: 28, 107, 178, 201, 266, 370, 373, 391, 415, 437, 464, 494, 518, 535, 557, 571, 604, 625, 670, 679, 689, 785, 795
15. This data was then validated by pasting original data into visual studio code and getting the same number of rows.

In [35]:
df2019 = pd.read_csv('data/cao2019_20211130_205230_edited.csv', sep='\t')

In [36]:
df2019

,Course Code,Course,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


In [37]:
# Adding a column to the dataframe specifying the course level, here level 8
df2019['Level'] = 8

In [38]:
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0,8
1,AL802,Software Design with Cloud Computing,301,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329,442.0,8
4,AL810,Quantity Surveying,307,349.0,8
...,...,...,...,...,...
925,WD200,Arts (options),221,296.0,8
926,WD210,Software Systems Development,271,329.0,8
927,WD211,Creative Computing,275,322.0,8
928,WD212,Recreation and Sport Management,274,311.0,8


### Level 6 & 7 data

#### In order to reproduce the data these are the steps I followed 
1. Download the original level 6 & 7 PDF file from the CAO website (link above). 
2. Open the original file using Microsoft Word. 
3. When open in Word, save the document as a word document. 
4. Re-save the Word document for editing.
5. Delete headers and page numbers in footer.
6. Select all in document and copy. 
7. Paste into Notepad++.
8. Remove institution name lines and blank lines.
9. In column headings, replace 'COURSE AND INSTITUTION' with 'Course', save file into folder as a csv file.
10. Remove tab after 'Mid' first line.
11. Delete tabs at end of lines: 31, 67, 69, 104, 108, 124, 157, 158, 174, 220, 224, 225, 228, 249, 294, 295, 328, 331, 365, 366, 410 and 433. 
12. This data was then validated by pasting original data into visual studio code and getting the same number of rows.

In [39]:
df2019_67 = pd.read_csv('data/cao2019_67_20211203_222130_edited.csv', sep='\t')

In [40]:
df2019_67

,Course Code,Course,EOS,Mid
0,AL600,Software Design,205,306.0
1,AL601,Computer Engineering,196,272.0
2,AL602,Mechanical Engineering,258,424.0
3,AL604,Civil Engineering,252,360.0
4,AL630,Pharmacy Technician,306,366.0
...,...,...,...,...
456,WD188,Applied Health Care,206,339.0
457,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0
458,WD206,Electronic Engineering,191,322.0
459,WD207,Mechanical Engineering,179,330.0


In [41]:
# Adding a column to the dataframe specifying the course level, here level 6 and 7
df2019_67['Level'] = '6 or 7'

In [42]:
df2019_67

,Course Code,Course,EOS,Mid,Level
0,AL600,Software Design,205,306.0,6 or 7
1,AL601,Computer Engineering,196,272.0,6 or 7
2,AL602,Mechanical Engineering,258,424.0,6 or 7
3,AL604,Civil Engineering,252,360.0,6 or 7
4,AL630,Pharmacy Technician,306,366.0,6 or 7
...,...,...,...,...,...
456,WD188,Applied Health Care,206,339.0,6 or 7
457,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0,6 or 7
458,WD206,Electronic Engineering,191,322.0,6 or 7
459,WD207,Mechanical Engineering,179,330.0,6 or 7


### Combining both 2019 dataframes into one containing level 6, level 7 and level 8 data.

In [43]:
# Combine both dataframes
df2019 = pd.concat([df2019, df2019_67], ignore_index=True)
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0,8
1,AL802,Software Design with Cloud Computing,301,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329,442.0,8
4,AL810,Quantity Surveying,307,349.0,8
...,...,...,...,...,...
1386,WD188,Applied Health Care,206,339.0,6 or 7
1387,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0,6 or 7
1388,WD206,Electronic Engineering,191,322.0,6 or 7
1389,WD207,Mechanical Engineering,179,330.0,6 or 7


Manipulate 2019 data to take out # and * from column and create new columns. Just leave numbers in points col.

In [44]:
#df2019.replace()
df2019['EOS'].replace('#', '',regex=True)

0       304
1       301
2       309
3       329
4       307
       ... 
1386    206
1387    208
1388    191
1389    179
1390    195
Name: EOS, Length: 1391, dtype: object

In [45]:
# Find a record with a '#' in EOS column of data set, use this to test code below. 
df2019.loc[df2019['Course Code']=='AL763']

,Course Code,Course,EOS,Mid,Level
957,AL763,Graphic Design (portfolio),#626,820.0,6 or 7


In [46]:
# Find a record with a '*' in EOS column of data set, use this to test code below. 
df2019.loc[df2019['Course Code']=='CK201']

,Course Code,Course,EOS,Mid,Level
124,CK201,Commerce,465*,489.0,8


In [47]:
# https://stackoverflow.com/questions/44117326/how-can-i-remove-all-non-numeric-characters-from-all-the-values-in-a-particular
# Remove characters in EOS column that are no numeric. 
df2019['EOS'] = df2019['EOS'].str.extract('(\d+)', expand=False)

In [48]:
# Validate if this worked
df2019.loc[df2019['Course Code']=='CK201']

,Course Code,Course,EOS,Mid,Level
124,CK201,Commerce,465,489.0,8


In [49]:
# Validate if this worked
df2019.loc[df2019['Course Code']=='AL763']

,Course Code,Course,EOS,Mid,Level
957,AL763,Graphic Design (portfolio),626,820.0,6 or 7


In [50]:
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0,8
1,AL802,Software Design with Cloud Computing,301,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329,442.0,8
4,AL810,Quantity Surveying,307,349.0,8
...,...,...,...,...,...
1386,WD188,Applied Health Care,206,339.0,6 or 7
1387,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0,6 or 7
1388,WD206,Electronic Engineering,191,322.0,6 or 7
1389,WD207,Mechanical Engineering,179,330.0,6 or 7


In [51]:
df2019.dtypes

Course Code     object
Course          object
EOS             object
Mid            float64
Level           object
dtype: object

In [52]:
# Conver EOS column to float 
df2019['EOS'] = df2019['EOS'].astype(float)

In [53]:
df2019.dtypes

Course Code     object
Course          object
EOS            float64
Mid            float64
Level           object
dtype: object

In [54]:
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,8
1,AL802,Software Design with Cloud Computing,301.0,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329.0,442.0,8
4,AL810,Quantity Surveying,307.0,349.0,8
...,...,...,...,...,...
1386,WD188,Applied Health Care,206.0,339.0,6 or 7
1387,WD205,Molecular Biology with Biopharmaceutical Science,208.0,441.0,6 or 7
1388,WD206,Electronic Engineering,191.0,322.0,6 or 7
1389,WD207,Mechanical Engineering,179.0,330.0,6 or 7


The 2019 data does not contain the institution name so I will use the course code and 2020 data in order to retrieve these higher education institute (HEI) names. 

In [55]:
# taking first 2 characters in Course Code column and outputting to new column hei_code column of df
df2019['hei_code'] = df2019['Course Code'].str[:2]
df2019

,Course Code,Course,EOS,Mid,Level,hei_code
0,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,8,AL
1,AL802,Software Design with Cloud Computing,301.0,306.0,8,AL
2,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,8,AL
3,AL805,Network Management and Cloud Infrastructure,329.0,442.0,8,AL
4,AL810,Quantity Surveying,307.0,349.0,8,AL
...,...,...,...,...,...,...
1386,WD188,Applied Health Care,206.0,339.0,6 or 7,WD
1387,WD205,Molecular Biology with Biopharmaceutical Science,208.0,441.0,6 or 7,WD
1388,WD206,Electronic Engineering,191.0,322.0,6 or 7,WD
1389,WD207,Mechanical Engineering,179.0,330.0,6 or 7,WD


#### Save Pandas Dataframe

In [56]:
# Creating a file path for the pandas data.
path2019_csv = 'data/cao2019_' + nowstring + '.csv'

# Save pandas datafram to folder
df2019.to_csv(path2019_csv)

***
***
# Joining the three years of data together.
***
## Concat join

In [57]:
# Create df for course in each year
courses2021 = df2021_2[['Course Code', 'Course Title', 'HEI', 'hei_code']]
courses2021.columns = ['code', 'title', 'hei', 'hei_code']

courses2020 = df2020[['COURSE CODE2', 'COURSE TITLE', 'HEI', 'hei_code']]
courses2020.columns = ['code', 'title', 'hei', 'hei_code']

courses2019 = df2019[['Course Code', 'Course', 'hei_code']]
courses2019.columns = ['code', 'title', 'hei_code']
courses2019

,code,title,hei_code
0,AL801,Software Design with Virtual Reality and Gaming,AL
1,AL802,Software Design with Cloud Computing,AL
2,AL803,Software Design with Mobile Apps and Connected...,AL
3,AL805,Network Management and Cloud Infrastructure,AL
4,AL810,Quantity Surveying,AL
...,...,...,...
1386,WD188,Applied Health Care,WD
1387,WD205,Molecular Biology with Biopharmaceutical Science,WD
1388,WD206,Electronic Engineering,WD
1389,WD207,Mechanical Engineering,WD


In [58]:
# Combine all three years of courses
allcourses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)
allcourses

,code,title,hei,hei_code
0,AL605,Music and Instrument Technology,Athlone Institute of Technology,AL
1,AL630,Pharmacy Technician,Athlone Institute of Technology,AL
2,AL631,Dental Nursing,Athlone Institute of Technology,AL
3,AL632,Applied Science,Athlone Institute of Technology,AL
4,AL650,Business,Athlone Institute of Technology,AL
...,...,...,...,...
4301,WD188,Applied Health Care,NaN,WD
4302,WD205,Molecular Biology with Biopharmaceutical Science,NaN,WD
4303,WD206,Electronic Engineering,NaN,WD
4304,WD207,Mechanical Engineering,NaN,WD


In [59]:
# Sort values by code 
allcourses.sort_values('code')

,code,title,hei,hei_code
273,AC120,International Business,American College,AC
1451,AC120,International Business,American College,AC
3083,AC120,International Business,NaN,AC
1452,AC137,Liberal Arts,American College,AC
3084,AC137,Liberal Arts,NaN,AC
...,...,...,...,...
3844,WD230,Mechanical and Manufacturing Engineering,NaN,WD
2914,WD230,Mechanical and Manufacturing Engineering,Waterford Institute of Technology,WD
1448,WD230,Mechanical and Manufacturing Engineering,Waterford Institute of Technology,WD
1449,WD231,Early Childhood Care and Education,Waterford Institute of Technology,WD


In [60]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

,code,title,hei,hei_code
1451,AC120,International Business,American College,AC
1452,AC137,Liberal Arts,American College,AC
1454,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD
1457,AD204,Fine Art (portfolio),National College of Art and Design,AD
1458,AD211,Fashion Design (portfolio),National College of Art and Design,AD
...,...,...,...,...
2910,WD208,Manufacturing Engineering,Waterford Institute of Technology,WD
2911,WD210,Software Systems Development,Waterford Institute of Technology,WD
2912,WD211,Creative Computing,Waterford Institute of Technology,WD
2913,WD212,Recreation and Sport Management,Waterford Institute of Technology,WD


In [61]:
# Returns a copy of the data frame with duplciates removed.
allcourses.drop_duplicates()

,code,title,hei,hei_code
0,AL605,Music and Instrument Technology,Athlone Institute of Technology,AL
1,AL630,Pharmacy Technician,Athlone Institute of Technology,AL
2,AL631,Dental Nursing,Athlone Institute of Technology,AL
3,AL632,Applied Science,Athlone Institute of Technology,AL
4,AL650,Business,Athlone Institute of Technology,AL
...,...,...,...,...
4301,WD188,Applied Health Care,NaN,WD
4302,WD205,Molecular Biology with Biopharmaceutical Science,NaN,WD
4303,WD206,Electronic Engineering,NaN,WD
4304,WD207,Mechanical Engineering,NaN,WD


In [62]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated(subset=['code'])]

,code,title,hei,hei_code
1451,AC120,International Business,American College,AC
1452,AC137,Liberal Arts,American College,AC
1453,AD101,"First Year Art & Design (Common Entry,portfolio)",National College of Art and Design,AD
1454,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD
1455,AD103,Textile & Surface Design and Jewellery & Objec...,National College of Art and Design,AD
...,...,...,...,...
4301,WD188,Applied Health Care,NaN,WD
4302,WD205,Molecular Biology with Biopharmaceutical Science,NaN,WD
4303,WD206,Electronic Engineering,NaN,WD
4304,WD207,Mechanical Engineering,NaN,WD


In [63]:
# Returns a copy of the data frame with duplciates removed - based only on code.
# We must ignore index in order to not bring through index
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)
allcourses

,code,title,hei,hei_code
0,AL605,Music and Instrument Technology,Athlone Institute of Technology,AL
1,AL630,Pharmacy Technician,Athlone Institute of Technology,AL
2,AL631,Dental Nursing,Athlone Institute of Technology,AL
3,AL632,Applied Science,Athlone Institute of Technology,AL
4,AL650,Business,Athlone Institute of Technology,AL
...,...,...,...,...
1807,SG432,Environmental Protection,NaN,SG
1808,SG438,Applied Archaeology,NaN,SG
1809,TL712,Computing with Digital Media,NaN,TL
1810,TL719,Culinary Arts,NaN,TL


I will utilize the same logic as above in order to populated the null values for the HEI that were not brought over from 2019 dataframe. I will use the 2020 and 2021 data in order to retrive the HEi and join it on below. 

In [64]:
#Create an array just with courses
colleges2021 = df2021_2[['HEI', 'hei_code']]
colleges2021.columns = ['hei', 'hei_code']

colleges2020 = df2020[['HEI', 'hei_code']]
colleges2020.columns = ['hei', 'hei_code']

colleges2019 = df2019[['hei_code']]


# Combine all three years of courses
allcolleges = pd.concat([colleges2021, colleges2020, colleges2020], ignore_index=True)
allcolleges

# Sort values by code 
allcolleges.sort_values('hei_code')

# Finds all extra copies of duplicated rows.
allcolleges[allcolleges.duplicated()]

# Returns a copy of the data frame with duplciates removed.
allcolleges.drop_duplicates()

# Finds all extra copies of duplicated rows.
allcolleges[allcolleges.duplicated(subset=['hei_code'])]

# Returns a copy of the data frame with duplciates removed - based only on code.
# We must ignore index in order to not bring through index
allcolleges.drop_duplicates(subset=['hei_code'], inplace=True, ignore_index=True)

allcolleges.columns = ['hei2', 'hei_code']

allcolleges

,hei2,hei_code
0,Athlone Institute of Technology,AL
1,"Institute of Technology, Carlow",CW
2,"Carlow College, St. Patrick`s",PC
3,Cork Institute of Technology,CR
4,University College Cork (NUI),CK
5,American College,AC
6,CCT College Dublin,CT
7,Marino Institute of Education,CM
8,Dorset College,DS
9,Dublin Business School,DB


In [65]:
# Set the index to the code column.
allcolleges.set_index('hei_code', inplace=True)
allcolleges

,hei2
hei_code,
AL,Athlone Institute of Technology
CW,"Institute of Technology, Carlow"
PC,"Carlow College, St. Patrick`s"
CR,Cork Institute of Technology
CK,University College Cork (NUI)
AC,American College
CT,CCT College Dublin
CM,Marino Institute of Education
DS,Dorset College


In [66]:
# Set the index to the code column.
allcourses.set_index('hei_code', inplace=True)
allcourses

,code,title,hei
hei_code,,,
AL,AL605,Music and Instrument Technology,Athlone Institute of Technology
AL,AL630,Pharmacy Technician,Athlone Institute of Technology
AL,AL631,Dental Nursing,Athlone Institute of Technology
AL,AL632,Applied Science,Athlone Institute of Technology
AL,AL650,Business,Athlone Institute of Technology
...,...,...,...
SG,SG432,Environmental Protection,NaN
SG,SG438,Applied Archaeology,NaN
TL,TL712,Computing with Digital Media,NaN


In [67]:
allcourses = allcourses.join(allcolleges)
allcourses

,code,title,hei,hei2
hei_code,,,,
AC,AC120,International Business,American College,American College
AC,AC137,Liberal Arts,American College,American College
AD,AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,National College of Art and Design
AD,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,National College of Art and Design
AD,AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,National College of Art and Design
...,...,...,...,...
WD,WD231,Early Childhood Care and Education,Waterford Institute of Technology,Waterford Institute of Technology
WD,WD232,Business Information Systems,Waterford Institute of Technology,Waterford Institute of Technology
WD,WD139,Civil Engineering,Waterford Institute of Technology,Waterford Institute of Technology


In [68]:
# Rearranging dataframe
allcourses = allcourses[['code', 'title', 'hei2']]
allcourses.columns = ['code', 'title','hei']
allcourses.set_index('code', inplace=True)
allcourses

,title,hei
code,,
AC120,International Business,American College
AC137,Liberal Arts,American College
AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design
AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design
AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design
...,...,...
WD231,Early Childhood Care and Education,Waterford Institute of Technology
WD232,Business Information Systems,Waterford Institute of Technology
WD139,Civil Engineering,Waterford Institute of Technology


In [69]:
allcourses.reset_index()
allcourses

,title,hei
code,,
AC120,International Business,American College
AC137,Liberal Arts,American College
AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design
AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design
AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design
...,...,...
WD231,Early Childhood Care and Education,Waterford Institute of Technology
WD232,Business Information Systems,Waterford Institute of Technology
WD139,Civil Engineering,Waterford Institute of Technology


In [70]:
allcourses['hei'].isnull().sum(axis = 0)

190

In [71]:
df2019['hei_code'] = df2019['Course Code'].str[:2]
df2019

,Course Code,Course,EOS,Mid,Level,hei_code
0,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,8,AL
1,AL802,Software Design with Cloud Computing,301.0,306.0,8,AL
2,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,8,AL
3,AL805,Network Management and Cloud Infrastructure,329.0,442.0,8,AL
4,AL810,Quantity Surveying,307.0,349.0,8,AL
...,...,...,...,...,...,...
1386,WD188,Applied Health Care,206.0,339.0,6 or 7,WD
1387,WD205,Molecular Biology with Biopharmaceutical Science,208.0,441.0,6 or 7,WD
1388,WD206,Electronic Engineering,191.0,322.0,6 or 7,WD
1389,WD207,Mechanical Engineering,179.0,330.0,6 or 7,WD


In [72]:
allcourses['hei'].isnull()

code
AC120    False
AC137    False
AD101    False
AD102    False
AD103    False
         ...  
WD231    False
WD232    False
WD139    False
WD149    False
WD148    False
Name: hei, Length: 1812, dtype: bool

In [73]:
allcourses[allcourses['hei'].isna()]

,title,hei
code,,
BN101,Business,NaN
BN103,Business and Information Technology,NaN
BN104,Computing (Information Technology),NaN
BN107,Applied Social Studies in Social Care,NaN
BN108,Engineering (Common Entry with Award options),NaN
...,...,...
TA312,Computing,NaN
TA313,Information Technology Management,NaN
TA314,Pharmaceutical Science,NaN


In [74]:
array = allcourses['hei'].nunique()

In [75]:
print(allcourses['hei'].unique())

['American College' 'National College of Art and Design'
 'Athlone Institute of Technology' 'St. Angela`s College' nan
 'IBAT College Dublin' 'Irish College of Humanities & Applied Sciences'
 'University College Cork (NUI)' 'Marino Institute of Education'
 'Cork Institute of Technology' 'CCT College Dublin'
 'Institute of Technology, Carlow' 'Dublin Business School'
 'Dublin City University' 'Dundalk Institute of Technology'
 'Dun Laoghaire Institute of Art, Design and Technology'
 'University College Dublin (NUI)' 'Dorset College'
 'Galway-Mayo Institute of Technology' 'Galway Business School'
 'Griffith College' 'National University of Ireland, Galway'
 'ICD Business School' 'Limerick Institute of Technology'
 'University of Limerick' 'Letterkenny Institute of Technology'
 'Maynooth University' 'Mary Immaculate College'
 "Pontifical University, St Patrick's College"
 'National College of Ireland (NCI)'
 'St Nicholas Montessori College Ireland' 'Carlow College, St. Patrick`s'
 'RCSI U

In [76]:
dsfjbsdfchk = ijbfJS

NameError: name 'ijbfJS' is not defined

### Joining to the Points and Other Columns
I will first create a datframe with all three years of data, no matter if the course was run over the course of 2019, 2020 and 2021 and give an insight into the points across these years individually. I will will then analyze the points for courses that were in operation across the three years.  

In [ ]:
df2021_data = df2021_2[['Course Code', 'CATEGORY (ISCED Description)', 'R1 Points', 'R2 Points ', 'EOS Points','EOS Midpoints', 'Course Level', 'HEI']]
df2021_data.columns = ['code', 'category', 'r1_2021', 'r2_2021', 'EOS_2021', 'MID_2021', 'level', 'HEI']
df2021_data

In [ ]:
# Set the index to the code column.
df2021_data.set_index('code', inplace=True)

In [ ]:
df2021_data

In [ ]:
# Set the index to the code column.
allcourses.set_index('code', inplace=True)

In [ ]:
#allcourses = allcourses.join(df2021[['points_r1_2021']])
#allcourses

In [ ]:
df2020_data = df2020[['COURSE CODE2', 'R1 POINTS', 'R2 POINTS', 'EOS','EOS Mid-point']]
df2020_data.columns = ['code', 'r1_2020', 'r2_2020', 'EOS_2020', 'MID_2020']
df2020_data

In [ ]:
# Set the index to the code column.
df2020_data.set_index('code', inplace=True)
df2020_data

In [ ]:
df2020_data

In [ ]:
df2019_data = df2019[['Course Code', 'EOS', 'Mid']]
df2019_data.columns = ['code', 'EOS_2019', 'MID_2019']
df2019_data

In [ ]:
# Set the index to the code column.
df2019_data.set_index('code', inplace=True)
df2019_data

In [ ]:
# Validate if this worked
df2019_data.loc['AL763']

In [ ]:
# Join 2021 points to allcourses.
allcourses2 = allcourses.join(df2021_data)
allcourses2

In [ ]:
# Join 2020 points to allcourses.
allcourses3 = allcourses2.join(df2020_data)
allcourses3

In [ ]:
# Join 2019 points to allcourses.
allcourses_final = allcourses3.join(df2019_data)
allcourses_final

In [ ]:
 AL605

In [ ]:
# Validate if this worked
allcourses_final.loc['AL763']

---

## References 